<a href="https://colab.research.google.com/github/mampisarkar111/wisonet-colab-demo/blob/main/Copy_of_Wisonet_kernel_demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing WisoMIP Model Ensemble and Satellite (TES) Water Vapor Isotope Profiles Before- and After- Using Averaging Kernels

This notebook demonstrates how to compare modeled HDO (deuterated water vapor) profiles with satellite retrievals (e.g., TES) in a physically consistent way.  

It shows:
- How to read, clean, and subset model and satellite data
- How to convert specific humidity to HDO volume mixing ratio (VMR)
- How to apply satellite averaging kernels (AK) and a priori profiles to the model for fair comparison
- How to visualize and interpret the results, both spatially and vertically

**Key concepts:**
- **Averaging kernel (AK):** Represents how the satellite's retrieval "sees" the vertical structure i.e., its true vertical sensitivity.  
- **A priori profile:** The climatological/initial guess profile used in the retrieval; the AK smooths the difference between model and a priori.
- **AK smoothing:** Applying the AK to model output produces a "retrieved-like" version, directly comparable to the satellite product.

This workflow shows an example case using monthly WisoSAT TES and WisoMIP Ensemble global 5x5 datasets from 2006 for the model-observation comparison (see Rodgers, 2000; TES User Guide).


In [ ]:
# Download SWING3 2006 subset data (NetCDF format) from Box
!curl -L "https://rice.box.com/shared/static/bcoy3ob0dme3umpurqmf0p6o48bznkj1" -o SWING3_2006_subset.nc > /dev/null 2>&1

# Download TES monthly 5° x 5° gridded isotope data (filtered) for 2006 from Box
!curl -L "https://rice.box.com/shared/static/uuy9m15qc1p7s4wm1yrzfzxc6knx7hzw" -o TES_monthly_5deg_strict.nc > /dev/null 2>&1

# Install cartopy (no output)
!pip install -q cartopy


In [ ]:
# Data Loading and Setup

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox, HBox
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
from cartopy.io import DownloadWarning

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="xarray")
warnings.filterwarnings("ignore", category=DownloadWarning, module="cartopy")
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Load model (SWING3) and satellite (TES) datasets
ds = xr.open_dataset("SWING3_2006_subset.nc")           # Model output
sat = xr.open_dataset("TES_monthly_5deg_strict.nc")     # Satellite retrieval

# Common dimension arrays for grids and levels
lon = ds['lon'].values
lat = ds['lat'].values
p = ds['p'].values                     # Model pressure [hPa]
tes_lat = sat['lat'].values
tes_lon = sat['lon'].values
tes_p = sat['level'].values            # TES pressure [hPa]
tes_time = sat['time'].values


## Scientific Background

### 1. Specific Humidity to Volume Mixing Ratio (VMR)

Specific humidity ($q$) is converted to molar/volume mixing ratio ($q_\mathrm{VMR}$) as:
$$
q_\mathrm{VMR} = \frac{q / M_\mathrm{H_2O}}{(q / M_\mathrm{H_2O}) + ((1-q) / M_\mathrm{air})}
$$
where:
- $M_\mathrm{H_2O}$: molar mass of H$_2$O ($18.01528$ g/mol)
- $M_\mathrm{air}$: molar mass of dry air ($28.9647$ g/mol)




### 2. Converting to HDO VMR

Given the reference HDO/H$_2$O ratio in Vienna Standard Mean Ocean Water (VSMOW), $R_\mathrm{vsmow} = 3.1152 \times 10^{-4}$, and the $\delta D$ value in permil ($\text{dD}$):
$$
\mathrm{HDO_{VMR}} = q_\mathrm{VMR} \times R_\mathrm{vsmow} \times \left(1 + \frac{\text{dD}}{1000}\right)
$$
We convert model-based specific humidity ($q$) to volume mixing ratio ($q_{\mathrm{VMR}}$) in order to match the satellite data units.

- **Model output** is typically in specific humidity ($q$), which is the mass of water vapor per mass of air (kg/kg).
- **Satellite retrievals** (such as TES) provide the volume mixing ratio (VMR), which is the number of water vapor molecules per total air molecules (mol/mol).

**Converting $q$ to $q_{\mathrm{VMR}}$ ensures that both model and satellite data are in the same units, allowing for direct, apples-to-apples comparison and proper application of the satellite averaging kernel.**

### 3. Applying the Averaging Kernel (AK)

Satellite retrievals do not "see" the full vertical structure. They apply a smoothing described by the **averaging kernel** ($\mathbf{A}$), and are anchored to a prior profile ($x_a$).

Directly comparing a model profile to a satellite retrieval is not valid because the satellite retrieval is always "blurred" (smoothed) by its AK and influenced by its a priori. To accurately simulate what the satellite would "see" if the model were reality, you must **apply the AK and a priori to the model** before comparing.

Following the approach described in **Worden et al. (2011)**, we apply the averaging kernel as follows (in log space):

$$
\ln(x_\mathrm{smoothed}) = \ln(x_a) + \mathbf{A} \cdot \left[\ln(x_\mathrm{model}) - \ln(x_a)\right]
$$

where:
- $x_\mathrm{model}$ is the vertical model profile, interpolated to the satellite pressure grid,
- $x_a$ is the satellite retrieval a priori profile,
- $\mathbf{A}$ is the satellite averaging kernel matrix (describing vertical sensitivity),
- $\ln$ denotes the natural logarithm (applied elementwise).

**This process ensures that model and satellite data are compared on the same "vertical sensitivity" basis, providing a fair, physically consistent comparison.**

*Reference: Worden, J. et al. (2011), "Estimate of bias in Aura TES HDO/H₂O profiles from comparison of TES and in situ HDO/H₂O measurements at the Mauna Loa observatory," ACP, 11, 4491–4503, [link](https://www.atmos-chem-phys.net/11/4491/2011/).*



## Interactive HDO VMR Maps: Satellite vs Model

This interactive tool lets you compare spatial maps of HDO volume mixing ratio (VMR) from:

- **TES Satellite Retrievals**: Observed values at each grid point.
- **Model "True" Field**: WisoMIP simulated $q$, converted to HDO VMR units.
- **AK-Corrected Model**: The model output processed with the TES averaging kernel and a priori, simulating what the satellite would “see” if the model were reality.

**How to use:**  
- Select a **pressure level** (in hPa) and a **month** (1 = January, 12 = December) to view.
- The three maps update by themselves: left = TES, middle = raw model, right = AK-corrected model.

**Tip:**  
The AK-corrected model panel is the best comparison to TES. It answers:  
*If the satellite were looking at the model world, what would it see?*

---

*See “Scientific Background” above for formulas and details.*


In [ ]:
plevel_input = widgets.BoundedFloatText(value=900, min=200, max=1000, step=10, description='Level [hPa]:')
month_slider = widgets.IntSlider(min=1, max=12, step=1, value=1, description='Month')

def update_plot(month, plevel):
    m = month - 1
    last_day = calendar.monthrange(2006, month)[1]
    user_date_str = f'2006-{month:02d}-{last_day}'
    target_date = datetime.strptime(user_date_str, '%Y-%m-%d')

    dD = ds['dD'].values.astype(float)
    q = ds['q'].values.astype(float)
    dD[dD == -999] = np.nan
    q[q == -999] = np.nan

    # --- Use full global region for maps
    lat_mask = np.full_like(lat, True, dtype=bool)
    lon_mask = np.full_like(lon, True, dtype=bool)

    dD_eq = dD[m, :, lat_mask, :][:, :, lon_mask]
    q_eq = q[m, :, lat_mask, :][:, :, lon_mask]

    dD_mean = np.nanmean(np.nanmean(dD_eq, axis=2), axis=0)
    q_mean = np.nanmean(np.nanmean(q_eq, axis=2), axis=0)

    # --- Model HDO calculations (global)
    Rvsmow = 3.1152e-4
    M_air = 28.9647
    M_H2O = 18.01528
    q_vmr = (q_mean / M_H2O) / ((q_mean / M_H2O) + ((1 - q_mean) / M_air))
    HDO_vmr1 = q_vmr * Rvsmow * (1 + dD_mean / 1000)

    q_vmr_3d = (q / M_H2O) / ((q / M_H2O) + ((1 - q) / M_air))
    HDO_vmr1_3d = q_vmr_3d * Rvsmow * (1 + dD / 1000)
    HDO_vmr1_3d = HDO_vmr1_3d[m, :, :, :]

    # --- Find TES time index for this month
    if np.issubdtype(tes_time.dtype, np.datetime64):
        t_tes = np.argmin(np.abs(tes_time - np.datetime64(user_date_str)))
    else:
        base_date = datetime(2000, 1, 1)
        t_tes = np.argmin(np.abs(tes_time - (target_date - base_date).days))

    idx_tes = int(np.argmin(np.abs(tes_p - plevel)))
    idx_mod = int(np.argmin(np.abs(p - plevel)))

    # --- 2D global maps (TES, raw model)
    HDO_globe = sat['HDO_vmr'].values[t_tes, idx_tes, :, :]
    HDO_model = HDO_vmr1_3d[idx_mod, :, :]

    # --- Compute AK-corrected model global map (on TES grid) ---
    nlat_tes, nlon_tes = len(tes_lat), len(tes_lon)
    HDO_model_corr_2d = np.full((nlat_tes, nlon_tes), np.nan)
    for i in range(nlat_tes):
        for j in range(nlon_tes):
            model_ilat = np.argmin(np.abs(lat - tes_lat[i]))
            model_ilon = np.argmin(np.abs(lon - tes_lon[j]))
            x = HDO_vmr1_3d[:, model_ilat, model_ilon]
            a = sat['HDO_ConstraintVector'].values[t_tes, :, i, j]
            A = sat['AK_HDO'].values[t_tes, :, :, i, j]
            if np.all(np.isfinite(x)) and np.all(np.isfinite(a)) and np.all(np.isfinite(A)):
                ln_true = np.log(x)
                ln_apriori = np.log(a)
                ln_smoothed = ln_apriori + A @ (ln_true - ln_apriori)
                HDO_model_corr_2d[i, j] = np.exp(ln_smoothed[idx_tes])

    # --- Mask fill values
    HDO_globe[HDO_globe == -999] = np.nan
    HDO_model[HDO_model == -999] = np.nan
    HDO_model_corr_2d[HDO_model_corr_2d == -999] = np.nan

    # --- Longitude sorting for model grid
    lon_plot = ((lon + 180) % 360) - 180
    sort_idx = np.argsort(lon_plot)
    lon_plot_sorted = lon_plot[sort_idx]
    HDO_model_sorted = HDO_model[:, sort_idx]

    # --- Colorbar limits (include all 3 maps)
    combined = np.concatenate([
        HDO_globe[np.isfinite(HDO_globe)],
        HDO_model_sorted[np.isfinite(HDO_model_sorted)],
        HDO_model_corr_2d[np.isfinite(HDO_model_corr_2d)]
    ])
    vmin, vmax = np.nanpercentile(combined, 1), np.nanpercentile(combined, 99)
    levels = np.linspace(vmin, vmax, 21)

    fig = plt.figure(figsize=(22, 8))

    ax1 = fig.add_subplot(1, 3, 1, projection=ccrs.PlateCarree())
    cs1 = ax1.contourf(tes_lon, tes_lat, HDO_globe, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax1.coastlines()
    ax1.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax1.gridlines(draw_labels=True, linewidth=0.3)
    ax1.set_title(f'TES HDO VMR at ~{plevel} hPa')
    fig.colorbar(cs1, ax=ax1, orientation='horizontal', pad=0.05).set_label('TES HDO VMR')

    ax2 = fig.add_subplot(1, 3, 2, projection=ccrs.PlateCarree())
    cs2 = ax2.contourf(lon_plot_sorted, lat, HDO_model_sorted, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax2.coastlines()
    ax2.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax2.gridlines(draw_labels=True, linewidth=0.3)
    ax2.set_title(f'Model HDO VMR at ~{plevel} hPa')
    fig.colorbar(cs2, ax=ax2, orientation='horizontal', pad=0.05).set_label('Model HDO VMR')

    ax3 = fig.add_subplot(1, 3, 3, projection=ccrs.PlateCarree())
    cs3 = ax3.contourf(tes_lon, tes_lat, HDO_model_corr_2d, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax3.coastlines()
    ax3.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax3.gridlines(draw_labels=True, linewidth=0.3)
    ax3.set_title(f'AK-corrected Model HDO VMR at ~{plevel} hPa')
    fig.colorbar(cs3, ax=ax3, orientation='horizontal', pad=0.05).set_label('AK-corrected Model HDO VMR')

    plt.tight_layout()
    plt.show()

# Display widgets
ui = VBox([
    HBox([plevel_input, month_slider])
])
out = interactive_output(update_plot, {
    'plevel': plevel_input,
    'month': month_slider
})
display(ui, out)


## Advanced Interactive Exploration: Regional Profiles and Vertical Sensitivity

In this section, you can:

- **Select a specific latitude/longitude region** (using Lat Min/Max, Lon Min/Max) to compute spatial averages and focus your analysis.
- **Choose the vertical profile view:**  
  - *Model True*: Profile from the model, converted to HDO VMR.
  - *TES Retrieved*: The satellite-retrieved HDO VMR.
  - *TES Apriori*: The a priori (prior) profile used in the satellite retrieval.
  - *AK-applied*: The model profile smoothed by the TES averaging kernel, simulating what the satellite would observe if the model were reality.
- **Toggle log scaling** for easier visualization of the full dynamic range of HDO in the vertical.

The **profile plot** (bottom left) shows how satellite and model vertical structures compare over your chosen region and month.

The **AK matrix plot** (bottom center) visualizes the vertical sensitivity of the TES retrieval at each pressure level—the “blurring” effect applied to the model.

**Tips:**
- Start with a broad latitude/longitude range, then zoom into smaller regions (e.g., a box over the subtropics or tropics) to see regional differences.
- Change the Step View to see the impact of the averaging kernel and a priori.
- Log scale is especially useful to view both upper and lower troposphere on the same axis.

---

*For scientific details, see the “Scientific Background” section above.*


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import calendar
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox, HBox

# User controls for interactive exploration
plevel_input = widgets.BoundedFloatText(value=900, min=200, max=1000, step=10, description='Level [hPa]:')
month_slider = widgets.IntSlider(min=1, max=12, step=1, value=1, description='Month')
log_toggle = widgets.Checkbox(value=True, description='Log X-axis')
step_dropdown = widgets.Dropdown(options=['All', 'Model True vs TES', 'Model True vs TES vs Apriori'],
                                value='All', description='Step View:')
lat_min_input = widgets.BoundedFloatText(value=20, min=-90, max=90, step=1, description='Lat Min:')
lat_max_input = widgets.BoundedFloatText(value=40, min=-90, max=90, step=1, description='Lat Max:')
lon_min_input = widgets.BoundedFloatText(value=-70, min=-180, max=180, step=5, description='Lon Min:')
lon_max_input = widgets.BoundedFloatText(value=-40, min=-180, max=180, step=5, description='Lon Max:')

def update_plot(month, plevel, log_x, step_view, lat_min, lat_max, lon_min, lon_max):
    m = month - 1
    last_day = calendar.monthrange(2006, month)[1]
    user_date_str = f'2006-{month:02d}-{last_day}'
    target_date = datetime.strptime(user_date_str, '%Y-%m-%d')

    dD = ds['dD'].values.astype(float)
    q = ds['q'].values.astype(float)
    dD[dD == -999] = np.nan
    q[q == -999] = np.nan

    # --- Find TES time and pressure indices for this month/plevel ---
    if np.issubdtype(tes_time.dtype, np.datetime64):
        t_tes = np.argmin(np.abs(tes_time - np.datetime64(user_date_str)))
    else:
        base_date = datetime(2000, 1, 1)
        t_tes = np.argmin(np.abs(tes_time - (target_date - base_date).days))
    idx_tes = int(np.argmin(np.abs(tes_p - plevel)))
    idx_mod = int(np.argmin(np.abs(p - plevel)))

    # --- Lat/lon subsetting for profile
    lat_mask = (lat >= lat_min) & (lat <= lat_max)
    lon_wrapped = (lon + 360) % 360
    lon_min_wrapped = (lon_min + 360) % 360
    lon_max_wrapped = (lon_max + 360) % 360
    if lon_min_wrapped < lon_max_wrapped:
        lon_mask = (lon_wrapped >= lon_min_wrapped) & (lon_wrapped <= lon_max_wrapped)
    else:
        lon_mask = (lon_wrapped >= lon_min_wrapped) | (lon_wrapped <= lon_max_wrapped)

    dD_eq = dD[m, :, lat_mask, :][:, :, lon_mask]
    q_eq = q[m, :, lat_mask, :][:, :, lon_mask]

    dD_mean = np.nanmean(np.nanmean(dD_eq, axis=2), axis=0)
    q_mean = np.nanmean(np.nanmean(q_eq, axis=2), axis=0)

    # --- Model HDO calculations (all-lat/lon for maps, subregion for profile)
    Rvsmow = 3.1152e-4
    M_air = 28.9647
    M_H2O = 18.01528
    q_vmr = (q_mean / M_H2O) / ((q_mean / M_H2O) + ((1 - q_mean) / M_air))
    HDO_vmr1 = q_vmr * Rvsmow * (1 + dD_mean / 1000)


    # --- 2D global maps (TES, raw model)
    HDO_globe = sat['H2O_vmr'].values[t_tes, idx_tes, :, :]
    #HDO_model = HDO_vmr1_3d[idx_mod, :, :]

    q_vmr_3d = (q / M_H2O) / ((q / M_H2O) + ((1 - q) / M_air))
    HDO_vmr_model_3d = q_vmr_3d[m, :, :, :]  # (level, lat, lon)
    HDO_model = HDO_vmr_model_3d[idx_mod, :, :]


    # --- Apply TES error mask to TES field only ---
    Err_HDO_diag = sat['Err_HDO'].values[t_tes, idx_tes, idx_tes, :, :]      # [lat, lon]
    Err_H2O_diag = sat['Err_H2O'].values[t_tes, idx_tes, idx_tes, :, :]
    Err_HDO_H2O_diag = sat['Err_HDO_H2O'].values[t_tes, idx_tes, idx_tes, :, :]
    dD_nobs_level = sat['dD_nobs'].values[t_tes, idx_tes, :, :]

    err_hdo = np.sqrt(Err_HDO_diag)
    err_h2o = np.sqrt(Err_H2O_diag)
    cov_hdo_h2o = Err_HDO_H2O_diag
    err_lnratio = np.sqrt(err_hdo**2 + err_h2o**2 - 2 * cov_hdo_h2o)
    err_dD = 1000 * err_lnratio

    nobs_thresh = 2
    percentile_cut = 90
    valid_nobs = dD_nobs_level >= nobs_thresh
    err_dD_all = err_dD[valid_nobs]
    if err_dD_all.size > 0:
        err_thresh = np.percentile(err_dD_all, percentile_cut)
    else:
        err_thresh = np.nanmax(err_dD)
    tes_mask = valid_nobs & (err_dD <= err_thresh)
    HDO_globe[~tes_mask] = np.nan

    # --- Longitude sorting for model grid
    lon_plot = ((lon + 180) % 360) - 180
    sort_idx = np.argsort(lon_plot)
    lon_plot_sorted = lon_plot[sort_idx]
    HDO_model_sorted = HDO_model[:, sort_idx]

    # --- Compute AK-corrected model global map (on TES grid) ---
    nlat_tes, nlon_tes = len(tes_lat), len(tes_lon)
    HDO_model_corr_2d = np.full((nlat_tes, nlon_tes), np.nan)
    for i in range(nlat_tes):      # TES lat
        for j in range(nlon_tes):  # TES lon
            model_ilat = np.argmin(np.abs(lat - tes_lat[i]))
            model_ilon = np.argmin(np.abs(lon - tes_lon[j]))
            x = HDO_vmr1_3d[:, model_ilat, model_ilon]   # [level]
            a = sat['H2O_ConstraintVector'].values[t_tes, :, i, j]
            A = sat['AK_H2O'].values[t_tes, :, :, i, j]
            if np.all(np.isfinite(x)) and np.all(np.isfinite(a)) and np.all(np.isfinite(A)):
                ln_true = np.log(x)
                ln_apriori = np.log(a)
                ln_smoothed = ln_apriori + A @ (ln_true - ln_apriori)
                HDO_model_corr_2d[i, j] = np.exp(ln_smoothed[idx_tes])

    # --- Mask fill values
    HDO_globe[HDO_globe == -999] = np.nan
    HDO_model[HDO_model == -999] = np.nan
    HDO_model_corr_2d[HDO_model_corr_2d == -999] = np.nan

    # --- Colorbar limits (include all 3 maps)
    combined = np.concatenate([
        HDO_globe[np.isfinite(HDO_globe)],
        HDO_model_sorted[np.isfinite(HDO_model_sorted)],
        HDO_model_corr_2d[np.isfinite(HDO_model_corr_2d)]
    ])
    vmin, vmax = np.nanpercentile(combined, 1), np.nanpercentile(combined, 99)
    levels = np.linspace(vmin, vmax, 21)

    # --- Map figure: 3 horizontal panels (TES, Model, AK-corrected Model) ---
    fig = plt.figure(figsize=(22, 8))

    ax1 = fig.add_subplot(2, 3, 1, projection=ccrs.PlateCarree())
    cs1 = ax1.contourf(tes_lon, tes_lat, HDO_globe, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax1.coastlines()
    ax1.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax1.gridlines(draw_labels=True, linewidth=0.3)
    ax1.add_patch(plt.Rectangle((lon_min, lat_min), lon_max - lon_min, lat_max - lat_min,
                                linewidth=2, edgecolor='red', facecolor='none', transform=ccrs.PlateCarree()))
    ax1.set_title(f'TES HDO VMR at ~{plevel} hPa')
    fig.colorbar(cs1, ax=ax1, orientation='horizontal', pad=0.05).set_label('TES HDO VMR')

    ax2 = fig.add_subplot(2, 3, 2, projection=ccrs.PlateCarree())
    cs2 = ax2.contourf(lon_plot_sorted, lat, HDO_model_sorted, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax2.coastlines()
    ax2.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax2.gridlines(draw_labels=True, linewidth=0.3)
    ax2.add_patch(plt.Rectangle((lon_min, lat_min), lon_max - lon_min, lat_max - lat_min,
                                linewidth=2, edgecolor='red', facecolor='none', transform=ccrs.PlateCarree()))
    ax2.set_title(f'Model Ensemble HDO VMR at ~{plevel} hPa')
    fig.colorbar(cs2, ax=ax2, orientation='horizontal', pad=0.05).set_label('Model Ensemble HDO VMR')

    ax3 = fig.add_subplot(2, 3, 3, projection=ccrs.PlateCarree())
    cs3 = ax3.contourf(tes_lon, tes_lat, HDO_model_corr_2d, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax3.coastlines()
    ax3.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax3.gridlines(draw_labels=True, linewidth=0.3)
    ax3.add_patch(plt.Rectangle((lon_min, lat_min), lon_max - lon_min, lat_max - lat_min,
                                linewidth=2, edgecolor='red', facecolor='none', transform=ccrs.PlateCarree()))
    ax3.set_title(f'AK-corrected Model HDO VMR at ~{plevel} hPa')
    fig.colorbar(cs3, ax=ax3, orientation='horizontal', pad=0.05).set_label('AK-corrected Model HDO VMR')

    plt.tight_layout()
    plt.show()

# === Display Widgets ===
ui = VBox([
    HBox([plevel_input]),
    HBox([month_slider]),
    HBox([lat_min_input, lat_max_input, lon_min_input, lon_max_input]),
    HBox([log_toggle, step_dropdown])
])
out = interactive_output(update_plot, {
    'plevel': plevel_input,
    'month': month_slider,
    'log_x': log_toggle,
    'step_view': step_dropdown,
    'lat_min': lat_min_input,
    'lat_max': lat_max_input,
    'lon_min': lon_min_input,
    'lon_max': lon_max_input
})
display(ui, out)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import calendar
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox, HBox

# --- Widgets (unchanged) ---
plevel_input = widgets.BoundedFloatText(value=900, min=200, max=1000, step=10, description='Level [hPa]:')
month_slider = widgets.IntSlider(min=1, max=12, step=1, value=1, description='Month')
log_toggle = widgets.Checkbox(value=True, description='Log X-axis')
step_dropdown = widgets.Dropdown(options=['All', 'Model True vs TES', 'Model True vs TES vs Apriori'],
                                value='All', description='Step View:')
lat_min_input = widgets.BoundedFloatText(value=20, min=-90, max=90, step=1, description='Lat Min:')
lat_max_input = widgets.BoundedFloatText(value=40, min=-90, max=90, step=1, description='Lat Max:')
lon_min_input = widgets.BoundedFloatText(value=-70, min=-180, max=180, step=5, description='Lon Min:')
lon_max_input = widgets.BoundedFloatText(value=-40, min=-180, max=180, step=5, description='Lon Max:')

def update_plot(month, plevel, log_x, step_view, lat_min, lat_max, lon_min, lon_max):
    m = month - 1
    last_day = calendar.monthrange(2006, month)[1]
    user_date_str = f'2006-{month:02d}-{last_day}'
    target_date = datetime.strptime(user_date_str, '%Y-%m-%d')

    dD = ds['dD'].values.astype(float)
    q = ds['q'].values.astype(float)
    dD[dD == -999] = np.nan
    q[q == -999] = np.nan

    # --- Find TES time and pressure indices for this month/plevel ---
    if np.issubdtype(tes_time.dtype, np.datetime64):
        t_tes = np.argmin(np.abs(tes_time - np.datetime64(user_date_str)))
    else:
        base_date = datetime(2000, 1, 1)
        t_tes = np.argmin(np.abs(tes_time - (target_date - base_date).days))
    idx_tes = int(np.argmin(np.abs(tes_p - plevel)))
    idx_mod = int(np.argmin(np.abs(p - plevel)))

    # --- Fix longitude axes for TES and model, ensure strictly ascending ---
    global tes_lon, lon  # <-- Use your global variables (as in your script)
    tes_lon = ((tes_lon + 180) % 360) - 180
    lon = ((lon + 180) % 360) - 180

    # Ensure strictly ascending and get sorting indices
    tes_lon, tes_lon_sort_idx = np.unique(tes_lon, return_index=True)
    lon, lon_sort_idx = np.unique(lon, return_index=True)

    # Sort model arrays along longitude axis
    # (Assume shape: (month, level, lat, lon))
    dD = dD[..., lon_sort_idx]
    q = q[..., lon_sort_idx]
    # Also update other model fields as needed

    # --- Model: Compute VMRs, HDO/H2O ratio, δD ---
    Rvsmow = 3.1152e-4
    M_air = 28.9647
    M_H2O = 18.01528
    q_vmr_3d = (q / M_H2O) / ((q / M_H2O) + ((1 - q) / M_air))
    H2O_vmr_3d = q_vmr_3d
    HDO_vmr_3d = H2O_vmr_3d * Rvsmow * (1 + dD / 1000)
    H2O_vmr_3d = H2O_vmr_3d[m, :, :, :]   # (level, lat, lon)
    HDO_vmr_3d = HDO_vmr_3d[m, :, :, :]   # (level, lat, lon)
    ratio_model_3d = HDO_vmr_3d / H2O_vmr_3d
    dD_model_3d = 1000 * (ratio_model_3d / Rvsmow - 1)  # shape (level, lat, lon)

    # --- 1. TES δD map and mask ---
    dD_globe = sat['dD'].values[t_tes, idx_tes, :, :].copy()
    dD_globe[(dD_globe == -999) | (np.abs(dD_globe) > 1e3)] = np.nan

    # --- TES error + nobs mask (to be applied to all panels for comparison) ---
    Err_HDO_diag = sat['Err_HDO'].values[t_tes, idx_tes, idx_tes, :, :]
    Err_H2O_diag = sat['Err_H2O'].values[t_tes, idx_tes, idx_tes, :, :]
    Err_HDO_H2O_diag = sat['Err_HDO_H2O'].values[t_tes, idx_tes, idx_tes, :, :]
    dD_nobs_level = sat['dD_nobs'].values[t_tes, idx_tes, :, :]
    err_hdo = np.sqrt(Err_HDO_diag)
    err_h2o = np.sqrt(Err_H2O_diag)
    cov_hdo_h2o = Err_HDO_H2O_diag
    err_lnratio = np.sqrt(err_hdo**2 + err_h2o**2 - 2 * cov_hdo_h2o)
    err_dD = 1000 * err_lnratio
    nobs_thresh = 2
    percentile_cut = 90
    valid_nobs = dD_nobs_level >= nobs_thresh
    err_dD_all = err_dD[valid_nobs]
    if err_dD_all.size > 0:
        err_thresh = np.percentile(err_dD_all, percentile_cut)
    else:
        err_thresh = np.nanmax(err_dD)
    tes_mask = valid_nobs & (err_dD <= err_thresh)
    dD_globe[~tes_mask] = np.nan

    # --- 2. Model δD on TES grid, masked to TES coverage ---
    nlat_tes, nlon_tes = len(tes_lat), len(tes_lon)
    dD_model_on_tes_grid = np.full((nlat_tes, nlon_tes), np.nan)
    for i in range(nlat_tes):
        for j in range(nlon_tes):
            model_ilat = np.argmin(np.abs(lat - tes_lat[i]))
            model_ilon = np.argmin(np.abs(lon - tes_lon[j]))
            dD_model_on_tes_grid[i, j] = dD_model_3d[idx_mod, model_ilat, model_ilon]
    dD_model_on_tes_grid[~tes_mask] = np.nan

    # --- 3. AK-corrected Model δD (using HDO/H2O ratio, AK, priors) ---
    dD_model_corr_2d = np.full((nlat_tes, nlon_tes), np.nan)
    for i in range(nlat_tes):
        for j in range(nlon_tes):
            model_ilat = np.argmin(np.abs(lat - tes_lat[i]))
            model_ilon = np.argmin(np.abs(lon - tes_lon[j]))
            r_model = ratio_model_3d[:, model_ilat, model_ilon]   # [level]
            a_hdo = sat['HDO_ConstraintVector'].values[t_tes, :, i, j]
            a_h2o = sat['H2O_ConstraintVector'].values[t_tes, :, i, j]
            a_r = a_hdo / a_h2o  # a priori ratio
            AK_r = sat['AK_HDO_H2O'].values[t_tes, :, :, i, j]   # [level, level]
            if np.all(np.isfinite(r_model)) and np.all(np.isfinite(a_r)) and np.all(np.isfinite(AK_r)):
                ln_true = np.log(r_model)
                ln_apriori = np.log(a_r)
                ln_smoothed = ln_apriori + AK_r.T @ (ln_true - ln_apriori)
                ratio_smoothed = np.exp(ln_smoothed)
                dD_smoothed = 1000 * (ratio_smoothed / Rvsmow - 1)
                dD_model_corr_2d[i, j] = dD_smoothed[idx_tes]
    dD_model_corr_2d[~tes_mask] = np.nan

    # --- Colorbar limits from all three maps ---
    combined = np.concatenate([
        dD_globe[np.isfinite(dD_globe)],
        dD_model_on_tes_grid[np.isfinite(dD_model_on_tes_grid)],
        dD_model_corr_2d[np.isfinite(dD_model_corr_2d)]
    ])
    vmin, vmax = np.nanpercentile(combined, 10), np.nanpercentile(combined, 90)
    #levels = np.linspace(vmin, vmax, 50)

    # --- Set fixed color limits ---
    #vmin, vmax = -120, -75
    levels = np.linspace(vmin, vmax, 20)  # or 21, adjust as needed

    fig = plt.figure(figsize=(22, 8))

    # Panel 1: TES δD
    ax1 = fig.add_subplot(1, 3, 1, projection=ccrs.PlateCarree())
    cs1 = ax1.contourf(
        tes_lon, tes_lat, dD_globe, levels=levels, cmap='plasma',
        vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree(), extend='both'
    )
    ax1.coastlines(); ax1.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax1.set_title('TES δD (‰)')
    fig.colorbar(cs1, ax=ax1, orientation='horizontal', pad=0.05, label='δD (‰)')

    # Panel 2: Model δD, masked like TES
    ax2 = fig.add_subplot(1, 3, 2, projection=ccrs.PlateCarree())
    cs2 = ax2.contourf(
        tes_lon, tes_lat, dD_model_on_tes_grid, levels=levels, cmap='plasma',
        vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree(), extend='both'
    )
    ax2.coastlines(); ax2.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax2.set_title('Model δD (masked)')
    fig.colorbar(cs2, ax=ax2, orientation='horizontal', pad=0.05, label='δD (‰)')

    # Panel 3: AK-corrected Model δD
    ax3 = fig.add_subplot(1, 3, 3, projection=ccrs.PlateCarree())
    cs3 = ax3.contourf(
        tes_lon, tes_lat, dD_model_corr_2d, levels=levels, cmap='plasma',
        vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree(), extend='both'
    )
    ax3.coastlines(); ax3.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax3.set_title('AK-corrected Model δD')
    fig.colorbar(cs3, ax=ax3, orientation='horizontal', pad=0.05, label='δD (‰)')

    plt.tight_layout()
    plt.show()


# === Display Widgets ===
ui = VBox([
    HBox([plevel_input]),
    HBox([month_slider]),
    HBox([lat_min_input, lat_max_input, lon_min_input, lon_max_input]),
    HBox([log_toggle, step_dropdown])
])
out = interactive_output(update_plot, {
    'plevel': plevel_input,
    'month': month_slider,
    'log_x': log_toggle,
    'step_view': step_dropdown,
    'lat_min': lat_min_input,
    'lat_max': lat_max_input,
    'lon_min': lon_min_input,
    'lon_max': lon_max_input
})
display(ui, out)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import calendar
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox, HBox

# User controls for interactive exploration
plevel_input = widgets.BoundedFloatText(value=900, min=200, max=1000, step=10, description='Level [hPa]:')
month_slider = widgets.IntSlider(min=1, max=12, step=1, value=1, description='Month')
log_toggle = widgets.Checkbox(value=True, description='Log X-axis')
step_dropdown = widgets.Dropdown(options=['All', 'Model True vs TES', 'Model True vs TES vs Apriori'],
                                value='All', description='Step View:')
lat_min_input = widgets.BoundedFloatText(value=20, min=-90, max=90, step=1, description='Lat Min:')
lat_max_input = widgets.BoundedFloatText(value=40, min=-90, max=90, step=1, description='Lat Max:')
lon_min_input = widgets.BoundedFloatText(value=-70, min=-180, max=180, step=5, description='Lon Min:')
lon_max_input = widgets.BoundedFloatText(value=-40, min=-180, max=180, step=5, description='Lon Max:')

def update_plot(month, plevel, log_x, step_view, lat_min, lat_max, lon_min, lon_max):
    m = month - 1
    last_day = calendar.monthrange(2006, month)[1]
    user_date_str = f'2006-{month:02d}-{last_day}'
    target_date = datetime.strptime(user_date_str, '%Y-%m-%d')

    q = ds['q'].values.astype(float)
    q[q == -999] = np.nan

    if np.issubdtype(tes_time.dtype, np.datetime64):
        t_tes = np.argmin(np.abs(tes_time - np.datetime64(user_date_str)))
    else:
        base_date = datetime(2000, 1, 1)
        t_tes = np.argmin(np.abs(tes_time - (target_date - base_date).days))
    idx_tes = int(np.argmin(np.abs(tes_p - plevel)))
    idx_mod = int(np.argmin(np.abs(p - plevel)))

    lat_mask = (lat >= lat_min) & (lat <= lat_max)
    lon_wrapped = (lon + 360) % 360
    lon_min_wrapped = (lon_min + 360) % 360
    lon_max_wrapped = (lon_max + 360) % 360
    if lon_min_wrapped < lon_max_wrapped:
        lon_mask = (lon_wrapped >= lon_min_wrapped) & (lon_wrapped <= lon_max_wrapped)
    else:
        lon_mask = (lon_wrapped >= lon_min_wrapped) | (lon_wrapped <= lon_max_wrapped)

    q_eq = q[m, :, lat_mask, :][:, :, lon_mask]
    q_mean = np.nanmean(np.nanmean(q_eq, axis=2), axis=0)

    M_air = 28.9647
    M_H2O = 18.01528
    q_vmr = (q_mean / M_H2O) / ((q_mean / M_H2O) + ((1 - q_mean) / M_air))

    H2O_globe = sat['H2O_vmr'].values[t_tes, idx_tes, :, :]

    q_vmr_3d = (q / M_H2O) / ((q / M_H2O) + ((1 - q) / M_air))
    H2O_vmr_model_3d = q_vmr_3d[m, :, :, :]
    H2O_model = H2O_vmr_model_3d[idx_mod, :, :]

    Err_H2O_diag = sat['Err_H2O'].values[t_tes, idx_tes, idx_tes, :, :]
    dD_nobs_level = sat['dD_nobs'].values[t_tes, idx_tes, :, :]
    err_h2o = np.sqrt(Err_H2O_diag)
    err_dD = 1000 * err_h2o

    nobs_thresh = 2
    percentile_cut = 90
    valid_nobs = dD_nobs_level >= nobs_thresh
    err_dD_all = err_dD[valid_nobs]
    if err_dD_all.size > 0:
        err_thresh = np.percentile(err_dD_all, percentile_cut)
    else:
        err_thresh = np.nanmax(err_dD)
    tes_mask = valid_nobs & (err_dD <= err_thresh)
    H2O_globe[~tes_mask] = np.nan

    lon_plot = ((lon + 180) % 360) - 180
    sort_idx = np.argsort(lon_plot)
    lon_plot_sorted = lon_plot[sort_idx]
    H2O_model_sorted = H2O_model[:, sort_idx]

    nlat_tes, nlon_tes = len(tes_lat), len(tes_lon)
    H2O_model_corr_2d = np.full((nlat_tes, nlon_tes), np.nan)
    for i in range(nlat_tes):
        for j in range(nlon_tes):
            model_ilat = np.argmin(np.abs(lat - tes_lat[i]))
            model_ilon = np.argmin(np.abs(lon - tes_lon[j]))
            x = H2O_vmr_model_3d[:, model_ilat, model_ilon]
            a = sat['H2O_ConstraintVector'].values[t_tes, :, i, j]
            A = sat['AK_H2O'].values[t_tes, :, :, i, j]
            if np.all(np.isfinite(x)) and np.all(np.isfinite(a)) and np.all(np.isfinite(A)):
                ln_true = np.log(x)
                ln_apriori = np.log(a)
                ln_smoothed = ln_apriori + A @ (ln_true - ln_apriori)
                H2O_model_corr_2d[i, j] = np.exp(ln_smoothed[idx_tes])

    H2O_globe[H2O_globe == -999] = np.nan
    H2O_model[H2O_model == -999] = np.nan
    H2O_model_corr_2d[H2O_model_corr_2d == -999] = np.nan

    combined = np.concatenate([
        H2O_globe[np.isfinite(H2O_globe)],
        H2O_model_sorted[np.isfinite(H2O_model_sorted)],
        H2O_model_corr_2d[np.isfinite(H2O_model_corr_2d)]
    ])
    vmin, vmax = np.nanpercentile(combined, 1), np.nanpercentile(combined, 99)
    levels = np.linspace(vmin, vmax, 21)

    fig = plt.figure(figsize=(22, 8))

    ax1 = fig.add_subplot(2, 3, 1, projection=ccrs.PlateCarree())
    cs1 = ax1.contourf(tes_lon, tes_lat, H2O_globe, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax1.coastlines()
    ax1.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax1.gridlines(draw_labels=True, linewidth=0.3)
    ax1.add_patch(plt.Rectangle((lon_min, lat_min), lon_max - lon_min, lat_max - lat_min,
                                linewidth=2, edgecolor='red', facecolor='none', transform=ccrs.PlateCarree()))
    ax1.set_title(f'TES H₂O VMR at ~{plevel} hPa')
    fig.colorbar(cs1, ax=ax1, orientation='horizontal', pad=0.05).set_label('TES H₂O VMR')

    ax2 = fig.add_subplot(2, 3, 2, projection=ccrs.PlateCarree())
    cs2 = ax2.contourf(lon_plot_sorted, lat, H2O_model_sorted, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax2.coastlines()
    ax2.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax2.gridlines(draw_labels=True, linewidth=0.3)
    ax2.add_patch(plt.Rectangle((lon_min, lat_min), lon_max - lon_min, lat_max - lat_min,
                                linewidth=2, edgecolor='red', facecolor='none', transform=ccrs.PlateCarree()))
    ax2.set_title(f'Model Ensemble H₂O VMR at ~{plevel} hPa')
    fig.colorbar(cs2, ax=ax2, orientation='horizontal', pad=0.05).set_label('Model Ensemble H₂O VMR')

    ax3 = fig.add_subplot(2, 3, 3, projection=ccrs.PlateCarree())
    cs3 = ax3.contourf(tes_lon, tes_lat, H2O_model_corr_2d, levels=levels, cmap='viridis',
                       vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    ax3.coastlines()
    ax3.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax3.gridlines(draw_labels=True, linewidth=0.3)
    ax3.add_patch(plt.Rectangle((lon_min, lat_min), lon_max - lon_min, lat_max - lat_min,
                                linewidth=2, edgecolor='red', facecolor='none', transform=ccrs.PlateCarree()))
    ax3.set_title(f'AK-corrected Model H₂O VMR at ~{plevel} hPa')
    fig.colorbar(cs3, ax=ax3, orientation='horizontal', pad=0.05).set_label('AK-corrected Model H₂O VMR')

    plt.tight_layout()
    plt.show()

# === Display Widgets ===
ui = VBox([
    HBox([plevel_input]),
    HBox([month_slider]),
    HBox([lat_min_input, lat_max_input, lon_min_input, lon_max_input]),
    HBox([log_toggle, step_dropdown])
])
out = interactive_output(update_plot, {
    'plevel': plevel_input,
    'month': month_slider,
    'log_x': log_toggle,
    'step_view': step_dropdown,
    'lat_min': lat_min_input,
    'lat_max': lat_max_input,
    'lon_min': lon_min_input,
    'lon_max': lon_max_input
})
display(ui, out)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import calendar
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox, HBox

# === User controls ===
plevel_input = widgets.BoundedFloatText(value=900, min=200, max=1000, step=10, description='Level [hPa]:')
month_slider = widgets.IntSlider(min=1, max=12, step=1, value=1, description='Month')

# === Function ===
def update_plot(month, plevel):
    m = month - 1
    last_day = calendar.monthrange(2006, month)[1]
    user_date_str = f'2006-{month:02d}-{last_day}'
    target_date = datetime.strptime(user_date_str, '%Y-%m-%d')

    dD = ds['dD'].values.astype(float)
    q = ds['q'].values.astype(float)
    dD[dD == -999] = np.nan
    q[q == -999] = np.nan

    Rvsmow = 3.1152e-4
    M_air = 28.9647
    M_H2O = 18.01528

    q_vmr_3d = (q / M_H2O) / ((q / M_H2O) + ((1 - q) / M_air))
    HDO_vmr1_3d = q_vmr_3d * Rvsmow * (1 + dD / 1000)
    HDO_vmr1_3d = HDO_vmr1_3d[m, :, :, :]
    H2O_vmr_model_3d = q_vmr_3d[m, :, :, :]

    if np.issubdtype(tes_time.dtype, np.datetime64):
        t_tes = np.argmin(np.abs(tes_time - np.datetime64(user_date_str)))
    else:
        base_date = datetime(2000, 1, 1)
        t_tes = np.argmin(np.abs(tes_time - (target_date - base_date).days))

    idx_tes = int(np.argmin(np.abs(tes_p - plevel)))
    idx_mod = int(np.argmin(np.abs(p - plevel)))

    HDO_globe = sat['HDO_vmr'].values[t_tes, idx_tes, :, :]
    H2O_globe = sat['H2O_vmr'].values[t_tes, idx_tes, :, :]
    HDO_model = HDO_vmr1_3d[idx_mod, :, :]
    H2O_model = H2O_vmr_model_3d[idx_mod, :, :]

    HDO_model_corr_2d = np.full((len(tes_lat), len(tes_lon)), np.nan)
    H2O_model_corr_2d = np.full((len(tes_lat), len(tes_lon)), np.nan)

    for i in range(len(tes_lat)):
        for j in range(len(tes_lon)):
            ilat = np.argmin(np.abs(lat - tes_lat[i]))
            ilon = np.argmin(np.abs(lon - tes_lon[j]))

            if not (np.isfinite(HDO_globe[i, j]) and np.isfinite(H2O_globe[i, j])):
                continue

            x_hdo = HDO_vmr1_3d[:, ilat, ilon]
            a_hdo = sat['HDO_ConstraintVector'].values[t_tes, :, i, j]
            A_hdo = sat['AK_HDO'].values[t_tes, :, :, i, j]

            x_h2o = H2O_vmr_model_3d[:, ilat, ilon]
            a_h2o = sat['H2O_ConstraintVector'].values[t_tes, :, i, j]
            A_h2o = sat['AK_H2O'].values[t_tes, :, :, i, j]

            if np.all(np.isfinite([*x_hdo, *a_hdo.flatten(), *A_hdo.flatten(), *x_h2o, *a_h2o.flatten(), *A_h2o.flatten()])):
                ln_true_hdo = np.log(x_hdo)
                ln_apriori_hdo = np.log(a_hdo)
                ln_smoothed_hdo = ln_apriori_hdo + A_hdo @ (ln_true_hdo - ln_apriori_hdo)
                HDO_model_corr_2d[i, j] = np.exp(ln_smoothed_hdo[idx_tes])

                ln_true_h2o = np.log(x_h2o)
                ln_apriori_h2o = np.log(a_h2o)
                ln_smoothed_h2o = ln_apriori_h2o + A_h2o @ (ln_true_h2o - ln_apriori_h2o)
                H2O_model_corr_2d[i, j] = np.exp(ln_smoothed_h2o[idx_tes])

    dD_tes = 1000 * ((HDO_globe / (H2O_globe * Rvsmow)) - 1)
    dD_model = 1000 * ((HDO_model / (H2O_model * Rvsmow)) - 1)
    dD_model_corr = 1000 * ((HDO_model_corr_2d / (H2O_model_corr_2d * Rvsmow)) - 1)

    mask_tes = ~np.isfinite(HDO_globe) | ~np.isfinite(H2O_globe) | \
               ~np.isfinite(HDO_model_corr_2d) | ~np.isfinite(H2O_model_corr_2d) | \
               ~np.isfinite(dD_tes) | ~np.isfinite(dD_model_corr)
    HDO_globe[mask_tes] = np.nan
    H2O_globe[mask_tes] = np.nan
    HDO_model_corr_2d[mask_tes] = np.nan
    H2O_model_corr_2d[mask_tes] = np.nan
    dD_tes[mask_tes] = np.nan
    dD_model_corr[mask_tes] = np.nan

    mask_model = ~np.isfinite(HDO_model) | ~np.isfinite(H2O_model) | ~np.isfinite(dD_model)
    HDO_model[mask_model] = np.nan
    H2O_model[mask_model] = np.nan
    dD_model[mask_model] = np.nan

    lon_plot = ((lon + 180) % 360) - 180
    sort_idx = np.argsort(lon_plot)
    lon_plot_sorted = lon_plot[sort_idx]
    HDO_model_sorted = HDO_model[:, sort_idx]
    H2O_model_sorted = H2O_model[:, sort_idx]
    dD_model_sorted = dD_model[:, sort_idx]

    hdo_vals = np.concatenate([HDO_globe[np.isfinite(HDO_globe)], HDO_model_sorted[np.isfinite(HDO_model_sorted)], HDO_model_corr_2d[np.isfinite(HDO_model_corr_2d)]])
    h2o_vals = np.concatenate([H2O_globe[np.isfinite(H2O_globe)], H2O_model_sorted[np.isfinite(H2O_model_sorted)], H2O_model_corr_2d[np.isfinite(H2O_model_corr_2d)]])
    dd_vals = np.concatenate([dD_tes[np.isfinite(dD_tes)], dD_model_sorted[np.isfinite(dD_model_sorted)], dD_model_corr[np.isfinite(dD_model_corr)]])

    vmin_hdo, vmax_hdo = np.nanpercentile(hdo_vals, 1), np.nanpercentile(hdo_vals, 99)
    vmin_h2o, vmax_h2o = np.nanpercentile(h2o_vals, 1), np.nanpercentile(h2o_vals, 99)
    vmin_dd, vmax_dd = np.nanpercentile(dd_vals, 1), np.nanpercentile(dd_vals, 99)

    levels_hdo = np.linspace(vmin_hdo, vmax_hdo, 21)
    levels_h2o = np.linspace(vmin_h2o, vmax_h2o, 21)
    levels_dd = np.linspace(vmin_dd, vmax_dd, 21)

    fig = plt.figure(figsize=(22, 16))

    titles = ['TES', 'Model', 'AK-corrected Model']
    row_labels = ['HDO', 'H₂O', 'δD']
    datasets = [
        (HDO_globe, HDO_model_sorted, HDO_model_corr_2d),
        (H2O_globe, H2O_model_sorted, H2O_model_corr_2d),
        (dD_tes, dD_model_sorted, dD_model_corr)
    ]
    level_sets = [levels_hdo, levels_h2o, levels_dd]
    vminmax = [
        (vmin_hdo, vmax_hdo),
        (vmin_h2o, vmax_h2o),
        (-200, 0)   # <-- Manually set δD colorbar range here
    ]


    for row in range(3):
        for col in range(3):
            ax = fig.add_subplot(3, 3, row * 3 + col + 1, projection=ccrs.PlateCarree())
            data = datasets[row][col]
            cs = ax.contourf(tes_lon if col != 1 else lon_plot_sorted, tes_lat if col != 1 else lat,
                             data, levels=level_sets[row], cmap='viridis',
                             vmin=vminmax[row][0], vmax=vminmax[row][1], transform=ccrs.PlateCarree())
            ax.coastlines()
            ax.add_feature(cfeature.BORDERS, linewidth=0.5)
            ax.gridlines(draw_labels=True, linewidth=0.3)
            ax.set_title(f'{titles[col]} {row_labels[row]} at ~{plevel} hPa')
            fig.colorbar(cs, ax=ax, orientation='horizontal', pad=0.05).set_label(f'{row_labels[row]}')

    plt.tight_layout()
    plt.show()

# === Display ===
ui = VBox([
    HBox([plevel_input, month_slider])
])
out = interactive_output(update_plot, {
    'plevel': plevel_input,
    'month': month_slider
})
display(ui, out)
